In [1]:
import numpy as np 
import scipy as sp

import control as ct
from core.utility import Output_Con

In [2]:
A = np.array([[0, 1,0],
             [0, 0, 1],
             [0, -2, -3]])
B = np.array([[0],[0],[1]]).reshape(-1,1)
C = np.eye(3)

In [3]:
dt = 0.1

In [4]:
Ad = A*dt + np.eye(A.shape[0])
Bd = B*dt
Cd  = C

In [5]:
Con = ct.ctrb(A,B)
print("Rank for state Controllability :", np.linalg.matrix_rank(Con))

Out_con, Out_rank = Output_Con(A, B, C)
print("Rank for Output Controllability :", Out_rank)


Rank for state Controllability : 3
Rank for Output Controllability : 3


In [6]:
# import the libraries
from control_files.nmpc_osqp import NonlinearMPCController
from dynamics.learned_models_control.linear_dynamics import linear_Dynamics

In [7]:
linear_model = linear_Dynamics(sp.sparse.csc_matrix(Ad), sp.sparse.csc_matrix(Bd), Cd)

In [8]:
solver_settings = {}
solver_settings['gen_embedded_ctrl'] = False
solver_settings['warm_start'] = True
solver_settings['polish'] = True
solver_settings['polish_refine_iter'] = 3
solver_settings['scaling'] = True
solver_settings['adaptive_rho'] = False
solver_settings['check_termination'] = 25
solver_settings['max_iter'] = 1000
solver_settings['eps_abs'] = 1e-6
solver_settings['eps_rel'] = 1e-6
solver_settings['eps_prim_inf'] = 1e-4
solver_settings['eps_dual_inf'] = 1e-4
solver_settings['linsys_solver'] = 'qdldl'

In [9]:
# closed loop evaluevation parameters
traj_length = 2000
max_iter = 100
N = 1
t_eval = dt*np.arange(traj_length+1)
num_input = 1

In [10]:
Q_mpc = sp.sparse.diags([100,100,100])
QN_mpc = sp.sparse.diags([1e3,1e3,1e3])
R_mpc = 1*sp.sparse.eye(num_input)                                     # Actuation penalty matrix, trajectory generation
R0_mpc = sp.sparse.csc_matrix(np.zeros(num_input))

In [11]:

# Design trajectory:
x0 = np.array([-0.2, 0.1, 0.3])                   # Initial value, closed loop trajectory
set_pt = np.array([0, 0, 0])              # Desired final value, closed loop trajectory
x_ref = np.tile(set_pt.reshape(-1,1), (1, traj_length))
xmax = np.array([5, 5,5])                          # State constraints, trajectory generation
xmin = -xmax
umax = 10
umin = -umax

# Define initial solution for SQP algorithm:
x_init = np.linspace(x0, x0, N+1)
u_init = 1*np.ones((num_input, N)).T
z_init = x_init

In [12]:
controller_nmpc = NonlinearMPCController(linear_model, N, dt, umin, umax, xmin, xmax, Q_mpc, R_mpc, QN_mpc, solver_settings)
controller_nmpc.construct_controller(z_init, u_init,x_ref[:,0])

(6, 7) (1, 7) (6, 7)


In [13]:
print(controller_nmpc._osqp_A.shape)
print(controller_nmpc._osqp_l.shape)
print(controller_nmpc._osqp_P.shape)
print(controller_nmpc._osqp_q.shape)

(13, 7)
(13,)
(7, 7)
(7,)


In [14]:
print(controller_nmpc.nz)
print(controller_nmpc.nx)
print(controller_nmpc.nu)

3
3
1


NameError: name 'z0' is not defined